In [3]:
import os
os.chdir('/home/sagemaker-user/ml-ops/lab3')
os.listdir()

['.env-sample',
 '.gitignore',
 'Dockerfile',
 'Dockerfile.airflow',
 'Dockerfile.mlflow',
 'Dockerfile.ms',
 'Dockerfile.train',
 'README.md',
 'dags',
 'data',
 'docker-compose.yaml',
 'requirements-airflow.txt',
 'requirements.txt',
 'scripts',
 'src',
 'sm']

In [4]:
import importlib
import src.sm.processing.data_prep as dp

importlib.reload(dp)

dv = dp.build_training_dataset(data_version="debug")
dv

TypeError: build_training_dataset() missing 2 required positional arguments: 'input_dir' and 'output_dir'

In [1]:
# ETL 
import os
import sagemaker
from datetime import datetime, UTC
import uuid
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

def generate_run_id(prefix: str = "run") -> str:
    """Generate a unique run_id using UTC timestamp + short UUID."""
    timestamp = datetime.now(UTC).strftime("%Y-%m-%d-%H-%M-%S")
    short_uuid = uuid.uuid4().hex[:6]
    return f"{prefix}-{timestamp}-{short_uuid}"

# SageMaker session and execution role
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

# Explicit project bucket (no default bucket)
BUCKET = "mlops-project-sm"

# Project S3 root prefix (your objects live under s3://bucket/data/...)
S3_ROOT = "data"
RAW_PREFIX = f"{S3_ROOT}/raw"
PROCESSED_PREFIX = f"{S3_ROOT}/processed"

# Generate run_id for this processing run
run_id = generate_run_id("banking-prep")
print("Using run_id:", run_id)

# Input must point to a NON-empty S3 prefix
raw_input_s3 = f"s3://{BUCKET}/{RAW_PREFIX}/"

# Output for this run_id
processed_output_s3 = f"s3://{BUCKET}/{PROCESSED_PREFIX}/runs/{run_id}/"

# Absolute path to your processing script
script_path = os.path.abspath("processing/data_prep.py")
reqs_path = os.path.abspath("processing/requirements.txt")

print("Script path:", script_path)

processor = SKLearnProcessor(
    framework_version="1.2-1",
    role=role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name="banking-data-prep",
    sagemaker_session=sess,
)

processor.run(
    code=script_path,
    arguments=[
        "--data_version", "auto",
        "--input_dir", "/opt/ml/processing/input",
        "--output_dir", "/opt/ml/processing/output",
        "--run_id", run_id,
    ],
    inputs=[
        ProcessingInput(
            source=raw_input_s3,
            destination="/opt/ml/processing/input",
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output",
            destination=processed_output_s3,
        )
    ],
    logs=True,
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Using run_id: banking-prep-2025-12-29-17-05-46-7943a9
Script path: /home/sagemaker-user/ml-ops/lab3/src/sm/processing/data_prep.py


INFO:sagemaker:Creating processing-job with name banking-data-prep-2025-12-29-17-05-46-299


...............>>> INPUT_DIR:  /opt/ml/processing/input
>>> OUTPUT_DIR: /opt/ml/processing/output
>>> DATA VERSION: banking-prep-2025-12-29-17-05-46-7943a9
>>> Loading GOLDEN train from: /opt/ml/processing/input/historical/train.csv
    GOLDEN train shape: (10003, 2)
>>> Loading GOLDEN test  from: /opt/ml/processing/input/historical/test.csv
    GOLDEN test shape:  (3080, 2)
>>> FEEDBACK inference dir:   /opt/ml/processing/input/logs/inference
>>> FEEDBACK corrections dir: /opt/ml/processing/input/logs/corrections
    Found inference JSONL files:   13
    Found corrections JSONL files: 3
    Reading /opt/ml/processing/input/logs/inference/2025-12-22/batch_231355_dc83da00-77aa-4467-8d8d-8f6098bf9ad1.jsonl
    Reading /opt/ml/processing/input/logs/inference/2025-12-22/0017b824-1d6c-43d1-ad53-0f9ee619c385.jsonl
    Reading /opt/ml/processing/input/logs/inference/2025-12-22/003f4307-3740-4b25-bc00-095ac31cc7d7.jsonl
    Reading /opt/ml/processing/input/logs/inference/2025-12-22/batch_22585

In [10]:
import sagemaker
sess = sagemaker.Session()
print("SageMaker region:", sess.boto_region_name)

SageMaker region: us-east-1


In [8]:
import boto3
import re

BUCKET = "mlops-project-sm"

def resolve_data_uris(data_version: str | None):
    """
    data_version:
      - None / "latest" => use processed/latest/*
      - otherwise       => use processed/runs/<data_version>/*
    """
    if not data_version or data_version == "latest":
        train_s3 = f"s3://{BUCKET}/data/processed/latest/train_latest.parquet"
        test_s3  = f"s3://{BUCKET}/data/processed/latest/test_latest.parquet"
        return train_s3, test_s3, "latest"

    # Assume it's a run_id
    train_s3 = f"s3://{BUCKET}/data/processed/runs/{data_version}/train.parquet"
    test_s3  = f"s3://{BUCKET}/data/processed/runs/{data_version}/test.parquet"
    return train_s3, test_s3, data_version


In [9]:
DATA_VERSION = "latest"  # or конкретный run_id
train_s3, test_s3, effective_version = resolve_data_uris(DATA_VERSION)
print("Using data_version:", effective_version)
print(train_s3)
print(test_s3)


Using data_version: latest
s3://mlops-project-sm/data/processed/latest/train_latest.parquet
s3://mlops-project-sm/data/processed/latest/test_latest.parquet


In [10]:
import boto3

def get_latest_run_id():
    s3 = boto3.client("s3")
    prefix = "data/processed/runs/"

    paginator = s3.get_paginator("list_objects_v2")
    run_ids = set()

    for page in paginator.paginate(Bucket=BUCKET, Prefix=prefix, Delimiter="/"):
        for cp in page.get("CommonPrefixes", []):
            # e.g. data/processed/runs/banking-prep-2025-12-25-.../
            run_prefix = cp["Prefix"]
            run_id = run_prefix[len(prefix):].strip("/")

            # Optional: filter only your runs
            if run_id.startswith("banking-prep-"):
                run_ids.add(run_id)

    if not run_ids:
        raise RuntimeError(f"No runs found under s3://{BUCKET}/{prefix}")

    # Your run_id starts with timestamp; lexical sort works if format is consistent.
    # If not consistent, better sort by LastModified of a known file (more complex).
    return sorted(run_ids)[-1]


In [11]:
DATA_VERSION = None  # None => auto
if DATA_VERSION is None:
    DATA_VERSION = get_latest_run_id()

train_s3, test_s3, effective_version = resolve_data_uris(DATA_VERSION)
print("Auto-selected run_id:", effective_version)


Auto-selected run_id: banking-prep-2025-12-29-17-05-46-7943a9


In [4]:
# Train
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.inputs import TrainingInput
from datetime import datetime
import uuid
from pathlib import Path

def generate_run_id(prefix="train"):
    ts = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
    return f"{prefix}-{ts}-{uuid.uuid4().hex[:6]}"

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
print(role)

BUCKET = "mlops-project-sm"
run_id = generate_run_id("banking-train")
print("Training run_id:", run_id)

# Data produced by Processing Job (we use latest for now)
# train_s3 = f"s3://{BUCKET}/data/processed/latest/train_latest.parquet"
# test_s3  = f"s3://{BUCKET}/data/processed/latest/test_latest.parquet"

# Local paths in Studio
entry_point = os.path.abspath("src/sm/training/train.py")
reqs_local  = os.path.abspath("src/sm/training/requirements.txt")

print("Entry point:", entry_point)
print("Reqs:", reqs_local)

estimator = SKLearn(
    entry_point=entry_point,
    role=role,
    framework_version="1.2-1",
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name="banking-training",
    sagemaker_session=sess,
    hyperparameters={
        # We pass where requirements will be mounted in the container
        "requirements": "/opt/ml/input/data/requirements/requirements.txt",
        # Filenames inside channels (we name files below)
        "train_file": "train_latest.parquet",
        "test_file": "test_latest.parquet",
        "max_features": 50000,
        "C": 2.0,
    },
)

inputs = {
    # Channels "train" and "test" can point directly to a single file S3 URI
    "train": TrainingInput(train_s3, content_type="application/x-parquet"),
    "test": TrainingInput(test_s3, content_type="application/x-parquet"),

    # Requirements channel: we upload local requirements.txt into this channel
    # For this, easiest is to put requirements.txt into S3 first OR use sagemaker.Session().upload_data
}

# Upload requirements.txt to S3 (simple and reliable)
reqs_s3_prefix = f"code/training/requirements/{run_id}"
reqs_s3_uri = sess.upload_data(path=reqs_local, bucket=BUCKET, key_prefix=reqs_s3_prefix)
print("Uploaded requirements to:", reqs_s3_uri)

inputs["requirements"] = TrainingInput(reqs_s3_uri, content_type="text/plain")

# Launch training
estimator.fit(inputs=inputs, job_name=run_id, wait=True, logs=True)


arn:aws:iam::191072691166:role/ml-ops-SageMaker-ExecutionRole
Training run_id: banking-train-2025-12-29-17-15-41-05dcb0
Entry point: /home/sagemaker-user/ml-ops/lab3/src/sm/src/sm/training/train.py
Reqs: /home/sagemaker-user/ml-ops/lab3/src/sm/src/sm/training/requirements.txt


/tmp/ipykernel_304/1057601797.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:53                                                                                   │
│                                                                                                  │
│   50                                                                                             │
│   51 inputs = {                                                                                  │
│   52 │   # Channels "train" and "test" can point directly to a single file S3 URI                │
│ ❱ 53 │   "train": TrainingInput(train_s3, content_type="application/x-parquet"),                 │
│   54 │   "test": TrainingInput(test_s3, content_type="application/x-parquet"),                   │
│   55 │                                                                                           │
│   56 │   # Requirements channel: we upload local requirements.txt into this channel              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train_s3' is not defined